# Viterbi Algorithm Example

## Problem Introduction

This example shows an implementation of the Viterbi algorithm to solve a simple and classic example of a Hidden Markov Model (HMM). In this example, our observable states represent a person's physical state. The three states are "normal", "cold", and "dizzy". Our Hidden Markov Process is a two-state process indicating whether or not the person is "Healthy" or has a "Fever".

## Defining the models

First, we write the Markov model in terms of the states, transition probabilities, and emission probabilities for the various observations.

In [7]:
obs = ('normal', 'normal', 'normal', 'dizzy', 'dizzy', 'cold')
states = ('Healthy', 'Fever')
start_p = {'Healthy': 0.6, 'Fever': 0.4}
trans_p = {
   'Healthy' : {'Healthy': 0.7, 'Fever': 0.3},
   'Fever' : {'Healthy': 0.4, 'Fever': 0.6}
   }
emit_p = {
   'Healthy' : {'normal': 0.5, 'cold': 0.4, 'dizzy': 0.1},
   'Fever' : {'normal': 0.1, 'cold': 0.3, 'dizzy': 0.6}
   }

## Model diagram

![title](HMM_example.png)


## Viterbi Algorithm Function

The Viterbi algorithm is a dynamic programming algorithm that keeps a table of the likely Markov chains, ending in each state, at each time step. It begins by storing in the table, for time 0, the probability of starting with each starting state and emitting the starting observation.

In [8]:
V = [{}]
for st in states:
    V[0][st] = {"prob": start_p[st] * emit_p[st][obs[0]], "prev": None}

Then, for each time step, we compute the most likely path ending in each state, and then the probability of this path emitting the observation for that time step. In this sense, it is very similar to the dynamic programming algorithms for shortest path (Dijkstra and Bellman-Ford).

In [9]:
for t in range(1, len(obs)):
    V.append({})
    for st in states:
        max_tr_prob = max(V[t-1][prev_st]["prob"]*trans_p[prev_st][st] for prev_st in states)
        for prev_st in states:
            if V[t-1][prev_st]["prob"] * trans_p[prev_st][st] == max_tr_prob:
                max_prob = max_tr_prob * emit_p[st][obs[t]]
                V[t][st] = {"prob": max_prob, "prev": prev_st}
                break

Now, we look through the resulting table to find the most likely final state of the Markov chain

In [10]:
opt = []
max_prob = max(value["prob"] for value in V[-1].values())
previous = None
for st, data in V[-1].items():
    if data["prob"] == max_prob:
        opt.append(st)
        previous = st
        break

And use the table to find the previous steps

In [11]:
for t in range(len(V) - 2, -1, -1):
    opt.insert(0, V[t + 1][previous]["prev"])
    previous = V[t + 1][previous]["prev"]

And we can print the Markov chain

In [13]:
print 'The steps of states are ( ' + '-> '.join(opt) + ' ) with highest probability of %s' % max_prob

The steps of states are ( Healthy-> Healthy-> Healthy-> Fever-> Fever-> Fever ) with highest probability of 0.000428652
